# Explore Satellite PKL Structure

This notebook recursively explores the structure of `props_gals_Fbox_new.pkl`
to understand the data format and verify mask-image alignment.

In [ ]:
import pickle
import sys
from pathlib import Path
from typing import Any

import numpy as np
import matplotlib.pyplot as plt

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

# PKL file location
PKL_PATH = Path('/shares/feldmann.ics.mnf.uzh/Lucas/pNbody/satellites/fbox/props_gals_Fbox_new.pkl')

## 1. Load PKL and Show Basic Info

In [ ]:
print(f"Loading: {PKL_PATH}")
print(f"Size: {PKL_PATH.stat().st_size / 1024 / 1024:.2f} MB")

with open(PKL_PATH, 'rb') as f:
    data = pickle.load(f)

print(f"\nType: {type(data).__name__}")
print(f"Total galaxies: {len(data)}")
print(f"\nSample keys: {list(data.keys())[:10]}")

## 2. Recursive Structure Explorer

In [ ]:
def explore_structure(obj: Any, prefix: str = "", max_depth: int = 4, current_depth: int = 0) -> None:
    """Recursively display structure of nested dicts/arrays."""
    indent = "  " * current_depth
    
    if current_depth >= max_depth:
        print(f"{indent}{prefix}... (max depth reached)")
        return
    
    if isinstance(obj, dict):
        print(f"{indent}{prefix}dict ({len(obj)} keys)")
        for i, (k, v) in enumerate(obj.items()):
            if i < 3:  # Show first 3 keys
                explore_structure(v, f"[{repr(k)}]: ", max_depth, current_depth + 1)
            elif i == 3:
                print(f"{indent}  ... and {len(obj) - 3} more keys")
                break
    elif isinstance(obj, np.ndarray):
        print(f"{indent}{prefix}ndarray shape={obj.shape}, dtype={obj.dtype}")
    elif isinstance(obj, (list, tuple)):
        print(f"{indent}{prefix}{type(obj).__name__} len={len(obj)}")
        if len(obj) > 0:
            explore_structure(obj[0], "[0]: ", max_depth, current_depth + 1)
    elif obj is None:
        print(f"{indent}{prefix}None")
    else:
        val_repr = repr(obj)[:50]
        print(f"{indent}{prefix}{type(obj).__name__} = {val_repr}")

# Explore structure starting from one galaxy
sample_key = list(data.keys())[0]
print(f"=== Structure of data['{sample_key}'] ===")
explore_structure(data[sample_key], max_depth=5)

## 3. Detailed Field Analysis

In [ ]:
# Get one complete satellite record
gal_key = '11, eo'
sb_key = 'SBlim27'
sat_key = 'id1'

satellite = data[gal_key][sb_key][sat_key]

print(f"=== Satellite Fields for {gal_key}/{sb_key}/{sat_key} ===")
print()

for field, value in satellite.items():
    if isinstance(value, np.ndarray):
        print(f"{field:20s}: ndarray shape={value.shape}, dtype={value.dtype}")
        if field == 'seg_ids':
            print(f"                     First 3 coords: {value[:3].tolist()}")
    elif value is None:
        print(f"{field:20s}: None")
    else:
        print(f"{field:20s}: {type(value).__name__} = {value}")

## 4. Visualize Mask on FITS Image (Alignment Verification)

In [ ]:
import gzip
from astropy.io import fits

def load_fits_gz(filepath: Path) -> np.ndarray:
    """Load gzipped FITS file."""
    with gzip.open(filepath, 'rb') as f:
        with fits.open(f) as hdul:
            return hdul[0].data

# Select a random galaxy for verification
import random
random.seed(42)

# Pick a galaxy that has satellites
valid_keys = [k for k in data.keys() if any(data[k].values())]
test_key = random.choice(valid_keys)
gal_id, orient = test_key.replace(' ', '').split(',')

print(f"Selected galaxy: {gal_id}, orientation: {orient}")

# Load FITS image
fbox_root = Path('/shares/feldmann.ics.mnf.uzh/data/LSB_and_Satellites/fbox')
fits_path = fbox_root / 'sb_maps' / f'magnitudes-Fbox-{gal_id}-{orient}-VIS.fits.gz'

if fits_path.exists():
    sb_map = load_fits_gz(fits_path)
    print(f"FITS shape: {sb_map.shape}")
else:
    print(f"FITS not found: {fits_path}")
    sb_map = None

In [ ]:
if sb_map is not None:
    # Get satellites for this galaxy
    gal_data = data[test_key]
    
    # Find a threshold with satellites
    test_sb = None
    for sb_key, sats in gal_data.items():
        if isinstance(sats, dict) and len(sats) > 0:
            test_sb = sb_key
            break
    
    if test_sb:
        print(f"Using threshold: {test_sb}")
        print(f"Satellites: {list(gal_data[test_sb].keys())}")
        
        # Create visualization
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # 1. Original FITS (Asinh stretch)
        flux = np.power(10.0, (22.5 - sb_map) / 2.5)
        flux = np.nan_to_num(flux, nan=0, posinf=0, neginf=0)
        vmin, vmax = np.percentile(flux[flux > 0], [1, 99])
        stretched = np.arcsinh(0.1 * flux)
        stretched = (stretched - stretched.min()) / (stretched.max() - stretched.min())
        
        axes[0].imshow(stretched, cmap='gray', origin='lower')
        axes[0].set_title(f'FITS Image: Fbox-{gal_id}-{orient}-VIS')
        
        # 2. seg_map from first satellite
        first_sat = list(gal_data[test_sb].values())[0]
        seg_map = first_sat['seg_map']
        print(f"seg_map shape: {seg_map.shape}")
        
        axes[1].imshow(seg_map > 0, cmap='Reds', origin='lower', alpha=0.7)
        axes[1].set_title(f'seg_map (satellite 1)')
        
        # 3. Overlay: Image + All satellite masks
        axes[2].imshow(stretched, cmap='gray', origin='lower')
        
        colors = plt.cm.tab10(np.linspace(0, 1, 10))
        for i, (sat_id, sat_data) in enumerate(gal_data[test_sb].items()):
            seg = sat_data['seg_map']
            mask_overlay = np.ma.masked_where(seg == 0, seg)
            axes[2].imshow(mask_overlay, cmap='Reds', alpha=0.5, origin='lower')
            
            # Mark center
            x, y = sat_data['x'], sat_data['y']
            axes[2].plot(x, y, 'b+', markersize=10, markeredgewidth=2)
            axes[2].text(x + 20, y, sat_id, color='blue', fontsize=10)
        
        axes[2].set_title(f'Overlay: Image + {len(gal_data[test_sb])} satellites')
        
        plt.tight_layout()
        plt.show()
    else:
        print("No satellites found for this galaxy")

## 5. Structure Summary

### PKL Structure
```
props_gals_Fbox_new.pkl
├── Type: dict
├── Size: ~13 GB
└── Keys: '{galaxy_id}, {orientation}' (e.g., '11, eo', '11, fo')
    └── Value: dict
        └── Keys: 'SBlim{threshold}' (e.g., 'SBlim27', 'SBlim27.5')
            └── Value: dict
                └── Keys: 'id{N}' (e.g., 'id1', 'id2', 'id3')
                    └── Value: dict (satellite properties)
                        ├── x: float64              # Centroid X
                        ├── y: float64              # Centroid Y
                        ├── geo-x: float64          # Geometric centroid X
                        ├── geo-y: float64          # Geometric centroid Y
                        ├── seg_map: ndarray (2051, 2051) uint8  # Full segmentation map
                        ├── seg_ids: ndarray (N, 2) int64        # Pixel coordinates [y, x]
                        ├── dmin: float64           # Min distance
                        ├── dmax: float64           # Max distance
                        ├── dmed: float64           # Median distance
                        ├── gini: float64           # Gini coefficient
                        ├── axis_ratio: float64     # Axis ratio
                        ├── orientation_angle: float64
                        ├── area: uint64            # Pixel count
                        ├── mag_fltr: float64       # Magnitude (filter)
                        ├── sb_fltr: float64        # Surface brightness
                        ├── mag_g: float64          # g-band magnitude
                        ├── mag_r: float64          # r-band magnitude
                        ├── mag_g_nodust: float64
                        └── mag_r_nodust: float64
```

### Key Notes
- `seg_map` is a full 2051×2051 binary mask (same size as fbox VIS images)
- `seg_ids` contains [y, x] pixel coordinates of the segmented region
- Coordinates are in the original 2051×2051 space (need resize to 1024 or 1072)